In [1]:
import cv2
import os
import uuid
import random
import numpy as np
from matplotlib import pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
from tensorflow.keras.metrics import Precision, Recall

In [2]:
# Siamese L1 Distance class
class L1Dist(Layer):
    
    # Init method - inheritance
    def __init__(self, **kwargs):
        super().__init__()
       
    # Magic happens here - similarity calculation
    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)

In [3]:
def preprocess(file_path):
    
    # Read in image from file path
    byte_img = tf.io.read_file(file_path)
    # Load in the image 
    img = tf.io.decode_jpeg(byte_img)
    
    # Preprocessing steps - resizing the image to be 105x105x3
    img = tf.image.resize(img, (105,105))
    # Scale image to be between 0 and 1 
    img = img / 255.0

    # Return image
    return img

In [4]:
ANC_PATH = os.path.join('VerificationImages', 'Image') 

In [5]:
# Load the pre-trained Haar cascade classifier for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Establish a connection to the webcam
cap = cv2.VideoCapture(0)

capturing = False
capture_count = 0

while cap.isOpened(): 
    ret, frame = cap.read()
   
    # Cut down frame to 250x250px
    frame = frame[120:120+250,200:200+250, :]
        # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Detect faces in the grayscale frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Draw rectangles around the detected faces on the display frame
    display_frame = frame.copy()
    for (x, y, w, h) in faces:
        cv2.rectangle(display_frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

    # Display the frame with face detection rectangles
    cv2.imshow('Face Detection', display_frame)

    # Check if the 'v' key is pressed and a face is detected
    key = cv2.waitKey(30)
    if key & 0xFF == ord('v') and len(faces) > 0:
        capturing = True 

    # Capture screenshots when capturing is True
    if capturing and len(faces) > 0 and capture_count < 20:
        # Create the unique file path 
        imgname = os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1()))
        # Write out anchor image
        cv2.imwrite(imgname, frame)
        # Show image back to screen
        cv2.imshow('Image Collection', frame) 
        capture_count += 1
        cv2.waitKey(30)
    elif capture_count >= 20:
        capturing = False
        print("Capturing complete.")
        break

    # Check if the 'q' key is pressed to quit
    if key & 0xFF == ord('q'):
        break

# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()  

Capturing complete.


In [6]:
siamese_model = tf.keras.models.load_model('snn_kash.h5', 
                                   custom_objects={'L1Dist':L1Dist, 'BinaryCrossentropy':tf.losses.BinaryCrossentropy})

In [7]:
for image in os.listdir(os.path.join('VerificationImages', 'Image')):
    validation_img = os.path.join('VerificationImages', 'Image', image)
    print(validation_img) 

VerificationImages\Image\b786f10f-3de6-11ee-b3ce-3003c8e2f0f6.jpg
VerificationImages\Image\b79e81dd-3de6-11ee-baa0-3003c8e2f0f6.jpg
VerificationImages\Image\b7aee594-3de6-11ee-b0ce-3003c8e2f0f6.jpg
VerificationImages\Image\b7bf4477-3de6-11ee-9781-3003c8e2f0f6.jpg
VerificationImages\Image\b7cd9559-3de6-11ee-b9e5-3003c8e2f0f6.jpg
VerificationImages\Image\b7de0f03-3de6-11ee-9d48-3003c8e2f0f6.jpg
VerificationImages\Image\b7ee725a-3de6-11ee-9ec8-3003c8e2f0f6.jpg
VerificationImages\Image\b7ff1230-3de6-11ee-8fc6-3003c8e2f0f6.jpg
VerificationImages\Image\b80fa1f7-3de6-11ee-ac6f-3003c8e2f0f6.jpg
VerificationImages\Image\b822b20a-3de6-11ee-bdaf-3003c8e2f0f6.jpg
VerificationImages\Image\b8333008-3de6-11ee-813b-3003c8e2f0f6.jpg
VerificationImages\Image\b8417771-3de6-11ee-8788-3003c8e2f0f6.jpg
VerificationImages\Image\b8547061-3de6-11ee-bbd4-3003c8e2f0f6.jpg
VerificationImages\Image\b864d865-3de6-11ee-9555-3003c8e2f0f6.jpg
VerificationImages\Image\b8733ff2-3de6-11ee-b17c-3003c8e2f0f6.jpg
Verificati

In [8]:
def verify(model, detection_threshold, verification_threshold):
    # Build results array
    results = []
    for image in os.listdir(os.path.join('VerificationImages', 'Image')):
        input_img = preprocess(os.path.join('VerificationImages', 'InputImage', 'input_image.jpg'))
        validation_img = preprocess(os.path.join('VerificationImages', 'Image', image))
        
        # Make Predictions 
        result = model.predict(list(np.expand_dims([input_img, validation_img], axis=1)))
        results.append(result)
    
    # Detection Threshold: Metric above which a prediciton is considered positive 
    detection = np.sum(np.array(results) > detection_threshold)
    
    # Verification Threshold: Proportion of positive predictions / total positive samples 
    verification = detection / len(os.listdir(os.path.join('VerificationImages', 'Image'))) 
    verified = verification > verification_threshold
    
    return results, verified 

In [9]:
# Create the "VerificationImage/InputImage" folders if they don't exist
if not os.path.exists("VerificationImages/InputImage"):
    os.makedirs("VerificationImages/InputImage")

# Load the pre-trained Haar cascade classifier for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Open the default camera (camera index 0)
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        print("Error: Could not read frame.")
        break

    frame_roi = frame[120:120+250, 200:200+250, :]
    
    # Display the verification frame
    cv2.imshow('Verification', frame_roi)

    key = cv2.waitKey(10) & 0xFF

    if key == ord('v'):
        input_image_path = os.path.join('VerificationImages', 'InputImage', 'input_image.jpg')
        cv2.imwrite(input_image_path, frame_roi)
        print(f"Input image saved as {input_image_path}")
    
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Input image saved as VerificationImages\InputImage\input_image.jpg
Input image saved as VerificationImages\InputImage\input_image.jpg


In [10]:
# Run verification using your 'verify' function and 'siamese_model'
results, verified = verify(siamese_model, 0.5, 0.5) 
print(verified) 

1/1 [==============================] - 0s 45ms/step
True
